In [57]:
from __future__ import print_function
import numpy as np
import pandas as pd

from keras.utils.np_utils import to_categorical

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Input, Embedding, LSTM, Dense,Dropout,GRU,Masking
from keras.models import Model
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from gensim.models import Word2Vec

In [58]:
train_x = pd.read_csv("modified.csv",encoding = 'latin1')

In [59]:
train_x=train_x['Comments']

In [60]:
np.shape(train_x)

(161693,)

In [61]:
train=train_x[0:5000]

In [62]:
np.shape(train)

(5000,)

In [63]:
for i in range(0,5000):
    train[i] = train[i].lower()

tokenizer = RegexpTokenizer(r'\w+')
for i in range(0,5000):
    train[i] = tokenizer.tokenize(train[i])


In [64]:
del tokenizer

In [65]:
count=0
for i in range(0,5000):
    if(len(train[i])<150):
        count=count+1
print(count)

4798


In [66]:
model_word2vec = Word2Vec(train,size =50,window = 5,min_count=1)

In [67]:
model_word2vec.wv.similar_by_word('confirm')

[('check', 0.9203464388847351),
 ('soon', 0.9002670645713806),
 ('close', 0.8994628190994263),
 ('come', 0.8921879529953003),
 ('thanks', 0.8729674816131592),
 ('let', 0.8701559901237488),
 ('whether', 0.8699354529380798),
 ('face', 0.8677573204040527),
 ('know', 0.8670606017112732),
 ('any', 0.863690197467804)]

In [68]:
model_word2vec.wv.get_vector('confirm')

array([  1.36780107e+00,   8.87938142e-01,   6.10094294e-02,
        -6.08940959e-01,  -9.90442395e-01,  -3.51829797e-01,
        -7.21544564e-01,   5.73705494e-01,  -4.37992811e-01,
         1.32294014e-01,   1.10031061e-01,  -4.34766747e-02,
         2.65548676e-01,   9.76784676e-02,   2.24685773e-01,
        -2.46469483e-01,   2.41752431e-01,  -5.09155571e-01,
         2.95402229e-01,   1.18601285e-01,  -1.80731460e-01,
         3.50163698e-01,  -8.84767652e-01,  -4.46130037e-01,
        -2.71982640e-01,  -1.03630461e-01,  -7.34228134e-01,
         1.70147407e+00,   1.17774203e-03,   1.96118340e-01,
         4.33391303e-01,  -3.23949188e-01,   2.01409012e-01,
         1.11269701e+00,  -4.15883362e-01,   4.77439076e-01,
         1.21172182e-01,   1.11821187e+00,   5.88089883e-01,
        -5.44902742e-01,   1.32156044e-01,   1.11213163e-01,
         5.32409310e-01,   1.25520504e+00,  -3.13499272e-02,
        -7.15162337e-01,   1.11216083e-01,   7.41250068e-02,
        -3.08896184e-01,

In [69]:
arr=np.zeros([5000,100,50])

In [70]:
b=np.zeros([50])
for i in range(0,5000):
    a = model_word2vec.wv[train[i]]
    if(len(a) >100):
        a = a[0:100]
    for j in range(0,(100-len(a))):
        a=np.vstack([a,b])
    arr[i] = a

del a 
del b

In [71]:
np.shape(arr)

(5000, 100, 50)

In [72]:
import tensorflow as tf

In [73]:
# The first part is unchanged
encoder_inputs = Input(shape=(100,50))
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
states = [state_h, state_c]

# Set up the decoder, which will only process one timestep at a time.
decoder_inputs = Input(shape=(1, 50))
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_dense = Dense(75,activation='relu')
decoder_dense2 = Dense(50)

In [74]:
from keras.layers import Lambda
from keras import backend as K

In [75]:
all_outputs = []
inputs = decoder_inputs
for _ in range(100):
    # Run the decoder on one timestep
    outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
    outputs1 = decoder_dense(outputs)
    outputs= decoder_dense2(outputs1)
    # Store the current prediction (we will concatenate all predictions later)
    all_outputs.append(outputs)
    # Reinject the outputs as inputs for the next loop iteration
    # as well as update the states
    inputs = outputs
    states = [state_h, state_c]

In [76]:
decoder_outputs.shape

TensorShape([Dimension(None), Dimension(100), Dimension(50)])

In [77]:
decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

In [78]:
model = Model([encoder_inputs, decoder_inputs],decoder_outputs)
model.compile(optimizer='adam', loss='mse')

In [79]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 100, 50)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1, 50)        0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 50), (None,  20200       input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 1, 50), (Non 20200       input_4[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [80]:
arr2 = np.zeros([5000,1,50])

In [81]:
model.fit([arr, arr2], arr, epochs=5,verbose=1)

Epoch 1/5
5000/5000 [==============================] - 141s 28ms/step - loss: 0.2122
Epoch 2/5
5000/5000 [==============================] - 44s 9ms/step - loss: 0.1632
Epoch 3/5
5000/5000 [==============================] - 49s 10ms/step - loss: 0.1526
Epoch 4/5
5000/5000 [==============================] - 45s 9ms/step - loss: 0.1502
Epoch 5/5
5000/5000 [==============================] - 46s 9ms/step - loss: 0.1461


In [82]:
xx = model.input[0]

In [83]:
yy=model.layers[2].get_output_at(node_index=0)[0]

In [84]:
yy

<tf.Tensor 'lstm_3/TensorArrayReadV3:0' shape=(?, 50) dtype=float32>

In [85]:
new_model=Model(xx,yy)

In [ ]:
s_in=arr[2]

In [87]:
np.shape(s_in)

(100, 50)

In [88]:
s_in=s_in.reshape(1,100,50)

In [89]:
res=new_model.predict(s_in)

In [90]:
tot_pred = new_model.predict(arr)

In [91]:
res

array([[ 0.574911  ,  0.59753197, -0.53574628, -0.40790498, -0.1401775 ,
        -0.5928458 , -0.14336608,  0.60426503, -0.60519105, -0.50327927,
         0.65367925,  0.17861417, -0.55071366, -0.45069546, -0.54768878,
         0.40572771, -0.50975043,  0.68396086, -0.48766878, -0.33699846,
        -0.57522041,  0.70693058,  0.24740291, -0.49150988,  0.7016713 ,
        -0.25025401,  0.18174851,  0.44395927, -0.45806846,  0.39528263,
         0.51075232,  0.62678772,  0.54429877,  0.63795471, -0.11536833,
        -0.55071837,  0.46870351, -0.14323892,  0.64036655,  0.65466416,
        -0.46140578, -0.19508725,  0.0070913 ,  0.59230542, -0.48492333,
         0.65206152, -0.01410156,  0.56190133, -0.5273155 , -0.6266225 ]], dtype=float32)

In [93]:
np.shape(tot_pred)

(5000, 50)

In [94]:
import pandas as pd

In [95]:
tot_pred=pd.DataFrame(tot_pred)

In [96]:
tot_pred.to_csv('reps.csv')